# Modeling

## Fix Seed and Set GPU

In [1]:
import torch
import random
import numpy as np
import os

seed = 50

os.environ['PYTHONHASHED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

In [2]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [3]:
device

device(type='cuda')

## Prepare Data

In [4]:
import pandas as pd

data_path = '/kaggle/input/plant-pathology-2020-fgvc7/'

DEBUG = False

if DEBUG:
    num_TTA = 1
    epochs = 1
else:
    num_TTA = 7
    epochs = 39

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [5]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(train,
                                test_size=0.1,
                               stratify=train[['healthy', 'multiple_diseases', 'rust', 'scab']],
                               random_state=50)

## Dataset

In [6]:
import cv2
from torch.utils.data import Dataset
import numpy as np

class ImageDataset(Dataset):
    def __init__(self, df, img_dir='./', transform=None, is_test=False):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]
        img_path = self.img_dir + img_id + '.jpg'
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform is not None:
            image = self.transform(image=image)['image']
        
        if self.is_test:
            return image
        else:
            label = np.argmax(self.df.iloc[idx, 1:5])
            return image, label      
        

In [7]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [8]:
transform_train = A.Compose([
    A.Resize(450, 650),
    A.RandomBrightnessContrast(brightness_limit=0.2,
                              contrast_limit=0.2, p=0.3),
    A.VerticalFlip(p=0.2),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(
    shift_limit=0.1,
    scale_limit=0.2,
    rotate_limit=30, p=0.3),
    A.OneOf([A.Emboss(p=1),
            A.Sharpen(p=1),
            A.Blur(p=1)], p=0.3),
    A.PiecewiseAffine(p=0.3),
    A.Normalize(),
    ToTensorV2()
])

In [9]:
transform_test = A.Compose([
    A.Resize(450, 650),
    A.Normalize(),
    ToTensorV2()
])

In [10]:
img_dir = '/kaggle/input/plant-pathology-2020-fgvc7/images/'

dataset_train = ImageDataset(df=train, img_dir=img_dir, transform=transform_train)
dataset_valid = ImageDataset(df=valid, img_dir=img_dir, transform=transform_test)

## Data Loader

In [11]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2 ** 32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [12]:
from torch.utils.data import DataLoader

batch_size = 4

loader_train = DataLoader(dataset_train, batch_size=batch_size,
                         shuffle=True, worker_init_fn=seed_worker,
                         generator=g, num_workers=2)
loader_valid = DataLoader(dataset_valid, batch_size=batch_size,
                         shuffle=False, worker_init_fn=seed_worker,
                         generator=g, num_workers=2)

## Load Pre-trained Model

In [13]:
!pip install efficientnet-pytorch==0.7.1

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=da19658b84063de3fac13593ee7562eee61a3d11a29146caf211d19978a145d0
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [14]:
from efficientnet_pytorch import EfficientNet

In [15]:
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=4)
model = model.to(device)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth


  0%|          | 0.00/254M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b7


## Train model and Validate

In [16]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00006, weight_decay=0.0001)

In [17]:
from transformers import get_cosine_schedule_with_warmup

scheduler = get_cosine_schedule_with_warmup(optimizer,
                                           num_warmup_steps=len(loader_train)*3,
                                           num_training_steps=len(loader_train)*epochs)

In [18]:
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm

for epoch in range(epochs):
    # 훈련
    model.train()
    epoch_train_loss = 0
    for images, labels in tqdm(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        epoch_train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        scheduler.step()
    print(f'에폭 [{epoch+1}/{epochs}] - 훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')
    
    # 검증
    model.eval()
    epoch_valid_loss = 0
    preds_list = []
    true_onehot_list = []
    
    with torch.no_grad():
        for images, labels in loader_valid:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            epoch_valid_loss += loss.item()
            
            preds = torch.softmax(outputs.cpu(), dim=1).numpy()
            true_onehot = torch.eye(4)[labels].cpu().numpy()
            
            preds_list.extend(preds)
            true_onehot_list.extend(true_onehot)
    print(f'에폭 [{epoch+1}/{epochs}] - 검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f} / 검증 데이터 ROC AUC : {roc_auc_score(true_onehot_list, preds_list):.4f}')

  0%|          | 0/410 [00:00<?, ?it/s]

에폭 [1/39] - 훈련 데이터 손실값 : 1.2793
에폭 [1/39] - 검증 데이터 손실값 : 0.6695 / 검증 데이터 ROC AUC : 0.9153


  0%|          | 0/410 [00:00<?, ?it/s]

에폭 [2/39] - 훈련 데이터 손실값 : 0.5753
에폭 [2/39] - 검증 데이터 손실값 : 0.2973 / 검증 데이터 ROC AUC : 0.9640


  0%|          | 0/410 [00:00<?, ?it/s]

...

에폭 [37/39] - 훈련 데이터 손실값 : 0.0051
에폭 [37/39] - 검증 데이터 손실값 : 0.1731 / 검증 데이터 ROC AUC : 0.9871


  0%|          | 0/410 [00:00<?, ?it/s]

에폭 [38/39] - 훈련 데이터 손실값 : 0.0028
에폭 [38/39] - 검증 데이터 손실값 : 0.1729 / 검증 데이터 ROC AUC : 0.9875


  0%|          | 0/410 [00:00<?, ?it/s]

에폭 [39/39] - 훈련 데이터 손실값 : 0.0047
에폭 [39/39] - 검증 데이터 손실값 : 0.1718 / 검증 데이터 ROC AUC : 0.9866


## Predict

In [19]:
dataset_test = ImageDataset(df=test, img_dir=img_dir,
                            transform=transform_test, is_test=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size,
                        shuffle=False, worker_init_fn=seed_worker,
                        generator=g, num_workers=2)

dataset_TTA = ImageDataset(df=test, img_dir=img_dir,
                          transform=transform_train, is_test=True)
loader_TTA = DataLoader(dataset_TTA, batch_size=batch_size,
                        shuffle=False, worker_init_fn=seed_worker,
                        generator=g, num_workers=2)

In [20]:
model.eval()

preds_test = np.zeros((len(test), 4))

with torch.no_grad():
    for i, images in enumerate(loader_test):
        images = images.to(device)
        outputs = model(images)
        preds_part = torch.softmax(outputs.cpu(), dim=1).squeeze().numpy()
        preds_test[i*batch_size:(i+1)*batch_size] += preds_part

In [21]:
submission_test = submission.copy()
submission_test.iloc[:, 1:5] = preds_test

## Predict with TTA

In [22]:
preds_tta = np.zeros((len(test), 4))

for i in range(num_TTA):
    with torch.no_grad():
        for i, images in enumerate(loader_TTA):
            images = images.to(device)
            outputs = model(images)
            preds_part = torch.softmax(outputs.cpu(), dim=1).squeeze().numpy()
            preds_tta[i*batch_size:(i+1)*batch_size] += preds_part

In [23]:
preds_tta /= num_TTA
submission_tta = submission.copy()
submission_tta.iloc[:, 1:5] = preds_tta

In [24]:
submission_test.to_csv('submission_test.csv', index=False)
submission_tta.to_csv('submmission_tta.csv', index=False)

## Predict with Label Smoothing

In [25]:
def apply_label_smoothing(df, target, alpha, threshold):
    df_target = df[target].copy()
    k = len(target)
    
    for idx, row in df_target.iterrows():
        if (row > threshold).any():
            row = (1 - alpha)*row + alpha/k
            df_target.iloc[idx] = row
    return df_target

In [26]:
alpha = 0.001
threshold = 0.999

submission_test_ls = submission_test.copy()
submission_tta_ls = submission_tta.copy()

target = ['healthy', 'multiple_diseases', 'rust', 'scab']

submission_test_ls[target] = apply_label_smoothing(submission_test_ls, target, alpha, threshold)
submission_tta_ls[target] = apply_label_smoothing(submission_tta_ls, target, alpha, threshold)

submission_test_ls.to_csv('submission_test_ls.csv', index=False)
submission_tta_ls.to_csv('submission_tta_ls.csv', index=False)